In [3]:
import pandas as pd
import os
from pymongo import MongoClient
from collections import defaultdict
from urllib.parse import quote_plus
import requests
#from obtener_tabla_ine import obtener_tabla_ine

In [4]:
import pprint

In [14]:
def obtener_tabla_ine(tabla_id, nult=None, tip=None, filtros=None):
    """
    Descarga cualquier tabla del INE vía API y la devuelve como DataFrame.
    - tabla_id: entero (p.ej. 6507)
    - nult: (opcional) sólo últimos N períodos
    - tip: (opcional) frecuencia ('A', 'M', …)
    - filtros: lista de tuplas [('Var1','Val1'), …]
    """
    url    = f"https://servicios.ine.es/wstempus/js/ES/DATOS_TABLA/{tabla_id}"
    params = {}
    if nult    is not None: params['nult'] = nult
    if tip     is not None: params['tip']  = tip
    if filtros:
        for var, val in filtros:
            params.setdefault('tv', []).append(f"{var}:{val}")

    resp = requests.get(url, params=params)
    resp.raise_for_status()
    data = resp.json()

    # Caso A: JSON “clásico” Tempus III con COLUMNS + Data
    if isinstance(data, dict) and 'COLUMNS' in data:
        # --- Igual que antes: extraemos cols dinámicas y aplastamos
        col_meta = data['COLUMNS']
        # intentar leer nombre de cada columna (dict o lista)
        dyn_cols = []
        for c in col_meta:
            if isinstance(c, dict) and 'VAR_NAME' in c:
                dyn_cols.append(c['VAR_NAME'])
            elif isinstance(c, (list,tuple)) and len(c) >= 2:
                dyn_cols.append(c[1])
            else:
                raise ValueError(f"Columna inesperada: {c!r}")
        # construimos un solo dataset estático
        registros = []
        for fila in data.get('Data', []):
            rec = dict(zip(dyn_cols, fila))
            registros.append(rec)
        return pd.DataFrame(registros)

    # Caso B: lista de series independientes (p.ej. tabla 6507)
    if isinstance(data, list) and data and isinstance(data[0], dict) and 'Data' in data[0]:
        registros = []
        for serie in data:
            # extraemos campos estáticos (todos menos 'Data')
            meta = {k: v for k, v in serie.items() if k != 'Data'}
            for periodo, valor in serie.get('Data', []):
                rec = dict(meta)            # COD, Nombre, FK_Unidad, …
                rec['Periodo'] = periodo    # nombre de tu elección
                rec['Valor']    = valor
                registros.append(rec)
        return pd.DataFrame(registros)

    # Si no es ninguno de los dos formatos, lanzar
    raise ValueError(f"Formato de respuesta INE inesperado: {type(data)}")

In [15]:
df_raw = obtener_tabla_ine_2(6507)
df_raw.head()

,COD,Nombre,FK_Unidad,FK_Escala
0,MNP162,Total Nacional. Nacimiento. Total. Todas las e...,120,1
1,MNP162,Total Nacional. Nacimiento. Total. Todas las e...,120,1
2,MNP162,Total Nacional. Nacimiento. Total. Todas las e...,120,1
3,MNP162,Total Nacional. Nacimiento. Total. Todas las e...,120,1
4,MNP162,Total Nacional. Nacimiento. Total. Todas las e...,120,1


In [16]:
df_raw.columns

Index(['COD', 'Nombre', 'FK_Unidad', 'FK_Escala'], dtype='object')

In [13]:
for columna in df_raw.columns: 
    print(df_raw[columna].unique())

['MNP162' 'MNP161' 'MNP160' 'MNP219' 'MNP218' 'MNP217' 'MNP216' 'MNP215'
 'MNP214' 'MNP213' 'MNP212' 'MNP211' 'MNP210' 'MNP209' 'MNP208' 'MNP207'
 'MNP206' 'MNP205' 'MNP204' 'MNP203' 'MNP202' 'MNP201' 'MNP200' 'MNP199'
 'MNP198' 'MNP197' 'MNP196' 'MNP195' 'MNP194' 'MNP193' 'MNP192' 'MNP191'
 'MNP190' 'MNP189' 'MNP188' 'MNP187' 'MNP186' 'MNP185' 'MNP184' 'MNP183'
 'MNP182' 'MNP181' 'MNP180' 'MNP179' 'MNP178' 'MNP177' 'MNP176' 'MNP175'
 'MNP174' 'MNP173' 'MNP172' 'MNP171' 'MNP170' 'MNP169' 'MNP168' 'MNP167'
 'MNP166' 'MNP165' 'MNP164' 'MNP163' 'MNP159' 'MNP158' 'MNP157']
['Total Nacional. Nacimiento. Total. Todas las edades. Total. Lugar de residencia de la madre. Dato base. '
 'Total Nacional. Nacimiento. Hombres. Todas las edades. Total. Lugar de residencia de la madre. Dato base. '
 'Total Nacional. Nacimiento. Mujeres. Todas las edades. Total. Lugar de residencia de la madre. Dato base. '
 'Andalucía. Nacimiento. Total. Todas las edades. Total. Lugar de residencia de la madre. Dato b